<a href="https://colab.research.google.com/github/pankaj-bhandari/spark/blob/main/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
 
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-2.3.1-bin-hadoop2.7"
 
import findspark
findspark.init()
 
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark

num = spark.sparkContext.textFile("/content/sample_data/california_housing_test.csv").count()
print(num)

num = spark.sparkContext.textFile("/content/sample_data/california_housing_test.csv")

num.take(10)


docs = spark.sparkContext.textFile("Important.txt").flatMap(lambda line: line.split(" ")).map(lambda word: (word,1)).reduceByKey(lambda a,b:a +b)

docs.take(10)



     |████████████████████████████████| 41 kB 63 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=917eb3cb8567c74faf91a60ddb66daf370e0c57c57a7ef50f6e915d47d59d0fc
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark
^C

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local

ModuleNotFoundError: ignored

In [ ]:
# import libraries
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
from datetime import timedelta, date
import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, HiveContext
from pyspark.sql import DataFrameStatFunctions as statFunc
from pyspark.sql.functions import regexp_extract, col

#spark settings
conf = SparkConf().setAppName('Smapi_myvf_es_pyspark')
#conf.set("spark.submit.deployMode","cluster")
conf.set("spark.dynamicAllocation.enabled", "true")
conf.set("spark.shuffle.service.enabled", "true")
conf.set("spark.dynamicAllocation.executorIdleTimeout", "2m")
conf.set("spark.dynamicAllocation.minExecutors", "1")
conf.set("spark.dynamicAllocation.maxExecutors", "1300")
conf.set("spark.rpc.io.serverThreads", "64")
conf.set("spark.memory.offHeap.enabled", "true")
conf.set("spark.memory.offHeap.size", "3g")
conf.set("spark.executor.memory", "3g")
conf.set("spark.yarn.executor.memoryOverhead", "3600")
conf.set("spark.shuffle.file.buffer","1MB")
conf.set("spark.unsafe.sorter.spill.reader.buffer.size","1MB")
conf.set("spark.file.transferTo","false")
conf.set("spark.io.compression.lz4.blockSize","512KB")
conf.set("spark.shuffle.service.index.cache.entries","2048")
#conf.set("spark.serializer","org.apache.spark.serializer.KryoSerializer")
#conf.set("spark.kryoserializer.buffer","1MB")
#conf.set("spark.default.parallelism","26")
#conf.set("spark.sql.shuffle.partitions",26)
#conf.set("spark.dynamicAllocation.initialExecutors","1")
#conf.set("spark.yarn.historyServer.address","10.109.214.142.18089")
#conf.set("spark.yarn.historyServer.allowTracking","true")

sc = SparkContext(conf=conf)
sqlContext=SQLContext(sc)

##### Data import #####
#yesterday_time=date.today() - timedelta(1) #put 1 here
#yesterday_date=yesterday_time.strftime("%Y%m%d")
#today_date=date.today().strftime("%Y%m%d")
for yesterday_date in 20190701,20190702,20190703,20190704,20190705:
	#read local json files
	field = [StructField('x_vf_net_type', StringType(), True),StructField('x_vf_screen_rotation', StringType(), True),
				StructField('x_vf_trace_source', StringType(), True),StructField('x_vf_trace_subject_region', StringType(), True),
				StructField('x_vf_trace_subject_id', StringType(), True),
				StructField('event_description', StringType(), True),
				StructField('x_vf_trace_session_id', StringType(), True),StructField('x_vf_trace_install_id', StringType(), True),
				StructField('x_vf_trace_source_version', StringType(), True),StructField('x_vf_trace_timestamp', StringType(), True),
				StructField('x_vf_trace_network_bearer', StringType(), True),StructField('x_vf_trace_user_agent', StringType(), True),
				StructField('x_vf_net_band', StringType(), True),StructField('x_vf_event_page', StringType(), True),
				StructField('x_vf_event_subpage', StringType(), True),StructField('x_vf_trace_platform', StringType(), True),
				StructField('x_vf_trace_os_version', StringType(), True),StructField('x_vf_trace_os_name', StringType(), True),
				StructField('x_vf_trace_application_name', StringType(), True),StructField('x_vf_trace_mcc', StringType(), True),
				StructField('x_vf_trace_mnc', StringType(), True),StructField('x_vf_trace_locale', StringType(), True),
				StructField('x_vf_trace_accept_language', StringType(), True),StructField('x_vf_event_type', StringType(), True),
				StructField('x_vf_event_element', StringType(), True),StructField('x_vf_trace_seclib_version', StringType(), True),
				StructField('sec_received_at', StringType(), True),StructField('x_vf_trace_host', StringType(), True),
				StructField('x_vf_trace_origin', StringType(), True),StructField('x_vf_screen_width', StringType(), True),
				StructField('x_vf_screen_height', StringType(), True),StructField('x_vf_trace_transaction_id', StringType(), True),
				StructField('event_description_dns', StringType(), True), StructField('event_description_duration', StringType(), True),
				StructField('event_description_httpCode', StringType(), True), StructField('event_description_redirect', StringType(), True),
				StructField('event_description_request', StringType(), True), StructField('event_description_response', StringType(), True),
				StructField('event_description_ssl', StringType(), True), StructField('event_description_start', StringType(), True),
				StructField('event_description_tcp', StringType(), True), StructField('event_description_type', StringType(), True),
				StructField('event_description_url', StringType(), True), StructField('host', StringType(), True),
				StructField('origin', StringType(), True), StructField('refer', StringType(), True), StructField('topic', StringType(), True)]
	schema = StructType(field)
	#in_dir = "/services/myvf_gr/in/"+yesterday_date+"/myvf-DE-js*"
	df = sqlContext.read.json("/services/myvf_GR/in/"+str(yesterday_date)+"/myvf-GR-js*",schema=schema)
	

	#standardization
	df = df.withColumnRenamed("x-vf-trace-height","x_vf_trace_height")\
			.withColumnRenamed("x-vf-trace-width","x_vf_trace_width")

	#Data analysis
	df = df.withColumn("sec_received_at",df.sec_received_at/1000)
	df = df.withColumn("x_vf_trace_timestamp",df.x_vf_trace_timestamp/1000)
	df = df.withColumn("sec_received_at", df["sec_received_at"].cast(IntegerType()))
	df = df.withColumn("x_vf_trace_timestamp", df["x_vf_trace_timestamp"].cast(IntegerType()))

	##Extracing the page_load
	df_filt1 = df.withColumn("page_load",f.when(df.x_vf_event_element.like("%page-loaded%") |\
												df.x_vf_event_element.like("%Load Time%"),f.regexp_extract('event_description', r"([0-9]+\.[0-9]+)",1)))
	df_filt1 = df_filt1.withColumn("page_load", df_filt1["page_load"].cast(FloatType()))

	##Extracing the response_time
	df_filt1 = df_filt1.withColumnRenamed("event_description_duration","response_time")

	##Extracing the response_code
	df_filt1 = df_filt1.withColumn("response_code",f.col("event_description_httpCode"))

	##Extracing the urls
	df_filt1 = df_filt1.withColumn("urls",f.regexp_extract('event_description_url', r"((http|https|sftp|ftp|file).{3})([\.a-zA-Z0-9\?\-\_\&\+\%\=]+)",3))
	df_filt1 = df_filt1.drop("event_description_url")

	##Extracing the browser
	df_filt1 = df_filt1.withColumn("browser",f.when(df_filt1.x_vf_trace_user_agent.like("%MSIE%") | df_filt1.x_vf_trace_user_agent.like("%Trident%"),"IE")
											  .when(df_filt1.x_vf_trace_user_agent.like("%Mozilla%") & (df_filt1.x_vf_trace_user_agent.like("%iPad%") |\
																										df_filt1.x_vf_trace_user_agent.like("%iPhone%") |\
																										df_filt1.x_vf_trace_user_agent.like("%Mac Os X%")),"Safari")        
											  .when(df_filt1.x_vf_trace_user_agent.like("%Chrome%") & df_filt1.x_vf_trace_user_agent.like("%Mobile Safari%"),"Chrome Mobile")
											  .when(df_filt1.x_vf_trace_user_agent.like("%Chrome%") & df_filt1.x_vf_trace_user_agent.like("%Safari%"),"Chrome")
											  .when(f.col("x_vf_trace_user_agent").isNull() | (f.col("x_vf_trace_user_agent") ==''),"")
											  .otherwise('Mozilla'))

	##Extracing the device_type
	df_filt1 = df_filt1.withColumn("device_type",f.when(f.lower(df_filt1.x_vf_trace_user_agent).like("%ipad%") |\
														f.lower(df_filt1.x_vf_trace_user_agent).like("%tablet%"),"Tablet")
												  .when(f.lower(df_filt1.x_vf_trace_user_agent).like("%mobile%") |\
														f.lower(df_filt1.x_vf_trace_user_agent).like("%iphone%") |\
														f.lower(df_filt1.x_vf_trace_user_agent).like("%android%") & ~f.lower(df_filt1.x_vf_trace_user_agent).like("%tablet%"),"Mobile")
												  .when(f.col("x_vf_trace_user_agent").isNull() | (f.col("x_vf_trace_user_agent") ==''),"")
												  .otherwise('Desktop'))

	##Extracing the platform
	df_filt1 = df_filt1.withColumn("platform",f.when(f.lower(df_filt1.x_vf_trace_platform).like("%android%"),"Android")
															.when(f.lower(df_filt1.x_vf_trace_platform).like("%ios%"),"iOS")
															.when(f.lower(df_filt1.x_vf_trace_platform).like("%js%"),"Web")
															.otherwise('N/A'))
	### extract os version ###
	df_filt1 = df_filt1.withColumn("os_version",f.concat(df_filt1.x_vf_trace_os_name, df_filt1.x_vf_trace_os_version))

	### change os name format ###
	df_filt1 = df_filt1.withColumn("os_name",f.when(df_filt1.x_vf_trace_os_name.like("M"),"Marshmallow (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("N"),"Nougat (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("Windows"),"Windows")
												.when(df_filt1.x_vf_trace_os_name.like("Linux"),"Linux")
												.when(df_filt1.x_vf_trace_os_name.like("N_MR1"),"Nougat_MR1 (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("KITKAT"),"KITKAT (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("L"),"Lollipop (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("Mac OS X"),"Mac OS X")
												.when(df_filt1.x_vf_trace_os_name.like("iOS"),"iOS")
												.when(df_filt1.x_vf_trace_os_name.like("Android"),"Android")
												.when(df_filt1.x_vf_trace_os_name.like("O_MR1"),"Oreo_MR1 (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("P"),"Pie (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("O"),"Oreo (Android)")
												.when(df_filt1.x_vf_trace_os_name.like("LOLLIPOP_MR1"),"Lollipop_MR1 (Android)"))

	### change network bearer format ###
	df_filt1 = df_filt1.withColumn("network_bearer",f.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("2g"),"2G")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("3g"),"3G")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("4g"),"4G")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("lte"),"LTE")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("bluetooth tethering"),"Bluetooth Tethering")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("ethernet"),"Ethernet")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("mobile"),"Mobile")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("slow-2g"),"slow-2g")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("vpn"),"VPN")
														.when(f.lower(df_filt1.x_vf_trace_network_bearer).like("wifi"),"WiFi"))

	##Extracing the satisfaction_level
	df_filt1 = df_filt1.withColumn("satisfaction_level",f.when(df_filt1.page_load<3,"Satisfied")
														 .when((df_filt1.page_load>=3) & (df_filt1.page_load<12),"Tolerated")
														 .when(df_filt1.page_load>=12,"Frustrated"))

	### extract subject region ###
	df_filt1 = df_filt1.withColumn("subject_region",f.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%al%"),"Albania")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ar%"),"Argentina")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%at%"),"Austria")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%be%"),"Belgium")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%br%"),"Brazil")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ca%"),"Canada")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ch%"),"Switzerland")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%co%"),"Colombia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%cy%"),"Cyprus")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%cz%"),"Czech")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%de%"),"Germany")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%dk%"),"Denmark")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ee%"),"Estonia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%eg%"),"Egypt")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%es%"),"Spain")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%fi%"),"Finland")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%fr%"),"France")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%gb%"),"Great Britain")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%gr%"),"Greece")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%hk%"),"Hong Kong")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%hr%"),"Croatia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%hu%"),"Hungary")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%id%"),"Indonesia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ie%"),"Ireland")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%il%"),"Israel")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%in%"),"India")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%iq%"),"Iraq")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ir%"),"Iran")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%is%"),"Iceland")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%it%"),"Italy")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%jp%"),"Japan")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%kp%"),"North Korea")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%kr%"),"South Korea")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%lu%"),"Luxembourg")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%mt%"),"Malta")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%mx%"),"Mexico")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%nl%"),"Netherlands")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%no%"),"Norway")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%np%"),"Nepal")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%nz%"),"New Zealand")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%pa%"),"Panama")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%pe%"),"Peru")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ph%"),"Philippines")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%pk%"),"Pakistan")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%pl%"),"Poland")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%pt%"),"Portugal")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%py%"),"Paraguay")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%qa%"),"Qatar")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ro%"),"Romania")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ru%"),"Russia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%sa%"),"Saudi Arabia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%se%"),"Sweden")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%sg%"),"Singapore")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%si%"),"Slovenia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%sk%"),"Slovakia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%sm%"),"San Marino")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%th%"),"Thailand")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%tn%"),"Tunisia")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%tr%"),"Turkey")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%tw%"),"Taiwan")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%ua%"),"Ukraine")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%us%"),"United States")
														.when(f.lower(df_filt1.x_vf_trace_subject_region).like("%za%"),"South Africa")
														.otherwise('Others'))

	##Extracing the tti
	df_filt1 = df_filt1.withColumn("tti",f.when(df_filt1.x_vf_event_element.like("%page-dom-loaded%"),f.regexp_extract('event_description', r"([0-9]+\.[0-9]+)",1))
													.otherwise(''))

	##Extracing the Tableau calculated field Failed_Requests                                     
	df_filt1 = df_filt1.withColumn("Failed_Requests",f.when((df_filt1["response_code"].cast(FloatType()) > 403) |\
															(df_filt1["response_code"].cast(FloatType()) == 400),1).otherwise(0))

	df_filt1 = df_filt1.drop("dt")

	##Extracing the session_duration
	df_min_max_copy = df_filt1.groupby(['platform','x_vf_trace_session_id']).agg(f.max('x_vf_trace_timestamp')\
																					.alias('max_session_timestamp'),
																				 f.min('x_vf_trace_timestamp')\
																					.alias('min_session_timestamp'))
	df_min_max_copy = df_min_max_copy.withColumn('session_duration',
												 (df_min_max_copy.max_session_timestamp-df_min_max_copy.min_session_timestamp))
	df_min_max_copy = df_min_max_copy.drop("max_session_timestamp")\
										.drop("min_session_timestamp")

	##Joining two DF (df_filt1,df_min_max_copy)
	df_filt1 = df_filt1.join(df_min_max_copy,['platform','x_vf_trace_session_id'],how='left')

	#memory help
	df_filt1.cache()

	#specify column types
	df_final = df_filt1.withColumn("tti", df_filt1["tti"].cast(FloatType()))
	#df_final = df_final.withColumn("page_load", df_final["page_load"].cast(FloatType()))
	df_final = df_final.withColumn("response_time", df_final["response_time"].cast(FloatType()))
	df_final = df_final.withColumn("session_duration", df_final["session_duration"].cast(FloatType()))

	##Logic to give_percentile
	def give_percentile(plat,metrics,perc):
		df_perc = df_final.filter(df_final.platform.like('%' + plat + '%'))
		df_perc = df_perc.filter(df_final[metrics]>0)
		df_perc.coalesce(1)
		percentile_val = df_perc.approxQuantile(metrics, [perc], 0.05)
		if len(percentile_val)>0:
			percentile_val = float(percentile_val[0])
		else:
			percentile_val = float(0)
		return round(percentile_val,5)

	##Creating DF to calling above functions
	df_agg = sqlContext.createDataFrame([Row(platform='Web',
												percentile_tti_50=give_percentile("Web","tti",0.5),
												percentile_tti_80=give_percentile("Web","tti",0.8),
												percentile_tti_90=give_percentile("Web","tti",0.9),
												percentile_tti_95=give_percentile("Web","tti",0.95),
												percentile_page_load_50=give_percentile("Web","page_load",0.5),
												percentile_page_load_80=give_percentile("Web","page_load",0.8),
												percentile_page_load_90=give_percentile("Web","page_load",0.9),
												percentile_page_load_95=give_percentile("Web","page_load",0.95),
												percentile_time_diff_50=give_percentile("Web","session_duration",0.5),
												percentile_time_diff_80=give_percentile("Web","session_duration",0.8),
												percentile_time_diff_90=give_percentile("Web","session_duration",0.9),
												percentile_time_diff_95=give_percentile("Web","session_duration",0.95),
												percentile_response_time_50=give_percentile("Web","response_time",0.5),
												percentile_response_time_80=give_percentile("Web","response_time",0.8),
												percentile_response_time_90=give_percentile("Web","response_time",0.9),
												percentile_response_time_95=give_percentile("Web","response_time",0.95))])

	##Taking date in format YYYY-MM-DD
	df_final = df_final.withColumn("sec_received_at",f.from_unixtime(df.sec_received_at,'yyyy-MM-dd')).cache() 

	##Extracing the isBounced
	df_max_dur_session = df_final.groupby(['x_vf_trace_session_id']).agg({'session_duration': 'max'})
	df_max_dur_session = df_max_dur_session.select(f.col('x_vf_trace_session_id'),
													f.col("max(session_duration)").alias("session_duration"))
	df_max_dur_session = df_max_dur_session.withColumn("isBounced",f.when(df_max_dur_session.session_duration < 10,1)) #.otherwise(0)
	df_max_dur_session = df_max_dur_session.drop("session_duration")

	#compute percentiles by x_vf_event_page
	percentiles=['0.5','0.8','0.9','0.95']
	list_df_p_event_page = []

	for p in percentiles:
		magic_percentile_tti = f.expr('percentile_approx(tti,'+ p +')')
		magic_percentile_resp_time = f.expr('percentile_approx(response_time,'+ p +')')
		df_p_event_page = df_final.groupBy(['platform','x_vf_event_page'])\
								  .agg(magic_percentile_tti.alias('percentile_tti_by_event_page_'+ str(int(float(p)*100))),
									   magic_percentile_resp_time.alias('percentile_response_time_by_event_page_'+ str(int(float(p)*100))))
		list_df_p_event_page.append(df_p_event_page)
			
	df_p_event_page = list_df_p_event_page[0]

	for df_next in list_df_p_event_page[1:]:
		df_p_event_page = df_p_event_page.join(df_next,['platform','x_vf_event_page'],how='inner')     

	#compute percentiles by url
	list_df_p_urls = []

	list_field_p = [[StructField('platform', StringType(), True),StructField('urls', StringType(), True),
					 StructField('percentile_tti_by_url_50', DoubleType(), True),
					 StructField('percentile_response_time_by_url_50', DoubleType(), True)],
					[StructField('platform', StringType(), True),StructField('urls', StringType(), True),
					 StructField('percentile_tti_by_url_80', DoubleType(), True),
					 StructField('percentile_response_time_by_url_80', DoubleType(), True)],
					[StructField('platform', StringType(), True),StructField('urls', StringType(), True),
					 StructField('percentile_tti_by_url_90', DoubleType(), True),
					 StructField('percentile_response_time_by_url_90', DoubleType(), True)],
					[StructField('platform', StringType(), True),StructField('urls', StringType(), True),
					 StructField('percentile_tti_by_url_95', DoubleType(), True),
					 StructField('percentile_response_time_by_url_95', DoubleType(), True)]]

	for p in percentiles:
		magic_percentile_tti = f.expr('percentile_approx(tti,'+ p +')')
		magic_percentile_resp_time = f.expr('percentile_approx(response_time,'+ p +')')
		df_p_urls = df_final.groupBy(['platform','urls']).agg(magic_percentile_tti.alias('percentile_tti_by_url_'+ str(int(float(p)*100))),\
															  magic_percentile_resp_time.alias('percentile_response_time_by_url_'+ str(int(float(p)*100))))
		list_df_p_urls.append(df_p_urls)
		
	list_df_p_urls_r = []

	cnt=0
	for df_t in list_df_p_urls:
		p_dir = '/services/dproduct/smapi_analytics/myvf_gr/out/cockpit/temp/tmp/gr/web/'+str(yesterday_date)+'/df_p_cross'+ str(cnt)
		df_t.write.format('json').mode("overwrite").save(p_dir) 
		cnt+=1
		
	for cnt in range(len(list_df_p_urls)):
		p_dir = '/services/dproduct/smapi_analytics/myvf_gr/out/cockpit/temp/tmp/gr/web/'+str(yesterday_date)+'/df_p_cross'+ str(cnt)
		df_r = sqlContext.read.json(p_dir,schema=StructType(list_field_p[cnt]))
		list_df_p_urls_r.append(df_r)
		
	df_p_urls = list_df_p_urls_r[0]
	for df_next in list_df_p_urls_r[1:]:
		df_p_urls = df_p_urls.join(df_next,['platform','urls'],how='inner')

	##Group by and same occurences count
	df_final_main =df_final.groupby(["platform"
	,"x_vf_trace_session_id"
	,"x_vf_trace_subject_id"
	,"subject_region"
	,"x_vf_trace_source_version"
	,"sec_received_at"
	,"network_bearer"
	,"x_vf_event_page"
	,"os_name"
	,"x_vf_event_type"
	,"x_vf_event_element"
	,"x_vf_trace_seclib_version"
	,"urls"
	,"browser"
	,"device_type"
	,"satisfaction_level"
	,"response_code"]).agg(
	f.count("response_time").cast(IntegerType()).alias("response_time_count"),
	f.sum("response_time").cast(FloatType()).alias("response_time_sum"),
	f.count("tti").cast(IntegerType()).alias("tti_count"),
	f.sum("tti").cast(FloatType()).alias("tti_sum"),
	f.count("page_load").cast(IntegerType()).alias("page_load_count"),
	f.sum("page_load").cast(FloatType()).alias("page_load_sum"),
	f.count("session_duration").cast(IntegerType()).alias("session_duration_count"),
	f.sum("session_duration").cast(FloatType()).alias("session_duration_sum"),
	f.sum("Failed_Requests").cast(FloatType()).alias("Failed_Requests_sum"),
	f.count("platform").cast(IntegerType()).alias("Total_non_agg_records") 
	)
	

	list_f = [StructField("platform",StringType(),True),
			  StructField("x_vf_trace_session_id",StringType(),True),
			  StructField("x_vf_trace_subject_id",StringType(),True),
			  StructField("subject_region",StringType(),False),
			  StructField("x_vf_trace_source_version",StringType(),True),
			  StructField("sec_received_at",StringType(),True),
			  StructField("network_bearer",StringType(),True),
			  StructField("x_vf_event_page",StringType(),True),
			  StructField("os_name",StringType(),True),
			  StructField("x_vf_event_type",StringType(),True),
			  StructField("x_vf_event_element",StringType(),True),
			  StructField("x_vf_trace_seclib_version",StringType(),True),
			  StructField("urls",StringType(),True),
			  StructField("browser",StringType(),False),
			  StructField("device_type",StringType(),False),
			  StructField("satisfaction_level",StringType(),False),
			  StructField("response_code",StringType(),True),
			  StructField("session_duration_count",IntegerType(),False),
			  StructField("session_duration_sum",FloatType(),True),
			  StructField("Failed_Requests_sum",FloatType(),True)]

	## write data to disk before join in order to make the job stable
	list_event_page_p =[StructField('x_vf_event_page',StringType(),True),
						StructField('platform',StringType(),False),
						StructField('percentile_tti_by_event_page_50',FloatType(),True),
						StructField('percentile_response_time_by_event_page_50',FloatType(),True),
						StructField('percentile_tti_by_event_page_80',FloatType(),True),
						StructField('percentile_response_time_by_event_page_80',FloatType(),True),
						StructField('percentile_tti_by_event_page_90',FloatType(),True),
						StructField('percentile_response_time_by_event_page_90',FloatType(),True),
						StructField('percentile_tti_by_event_page_95',FloatType(),True),
						StructField('percentile_response_time_by_event_page_95',FloatType(),True)]

	event_page_dir = '/services/dproduct/smapi_analytics/myvf_gr/out/cockpit/temp/tmp/gr/web/'+str(yesterday_date)+'/df_p_event_page'
	df_p_event_page.write.format('json').mode("overwrite").save(event_page_dir)
	df_p_event_page = sqlContext.read.json(event_page_dir ,schema=StructType(list_event_page_p))

	## join
	df_final_main = df_final_main.join(df_max_dur_session,['x_vf_trace_session_id'],how='left')
	df_final_main = df_final_main.join(df_agg,['platform'],how='left')
	df_final_main = df_final_main.join(df_p_event_page,['platform','x_vf_event_page'],how='left') 
	df_final_main = df_final_main.join(df_p_urls,['platform','urls'],how='left')

	##### Export Data #####
	df_final_main.write.mode("overwrite").option("maxRecordsPerFile", 1000000).parquet("/services/dproduct/smapi_analytics/myvf_gr/out/cockpit/web/dt="+str(yesterday_date))
	#out_dir = "/services/myvf_gr/out/web/dt="+yesterday_date
	#df_final_main.repartition(30).write.format("parquet").mode("overwrite").save("/services/dproduct/smapi_analytics/myvf_gr/out/cockpit/web/dt="+str(yesterday_date))




## More Resources

### Working with Notebooks in Colab
- [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)
- <img src="/img/new.png" height="20px" align="left" hspace="4px" alt="New"></img>
 [TensorFlow 2 in Colab](/notebooks/tensorflow_version.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb) 
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas](/notebooks/mlcc/intro_to_pandas.ipynb)
- [Tensorflow concepts](/notebooks/mlcc/tensorflow_programming_concepts.ipynb)

<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colaboratory makes possible, check out these  tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.
